## Importing Libraries

In [26]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd

# 1. Creating a dataset class

In [27]:
class BookDataset(Dataset):
    def __init__(self, features, transform=None):
        self.features = torch.FloatTensor(features)
        self.transform = transform
    
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        sample = self.features[idx]
        
        if self.transform:
            sample = self.transform(sample)
        
        return sample

# 2. Create the neural network model

In [28]:
class BookRecommenderNet(nn.Module):
    def __init__(self, input_size):
        super(BookRecommenderNet, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_size, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, input_size)
        )
    
    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# 3. Training function

In [29]:
def train_model(model, train_loader, num_epochs=100, learning_rate=0.001):
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    print("Training started...")
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch in train_loader:
            optimizer.zero_grad()
            encoded, decoded = model(batch)
            loss = criterion(decoded, batch)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
        if (epoch + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}')
    
    print("Training completed!")
    return model

# 4. Function to get book embeddings

In [30]:
def get_book_embeddings(model, features):
    model.eval()
    with torch.no_grad():
        features_tensor = torch.FloatTensor(features)
        encoded, _ = model(features_tensor)
    return encoded.numpy()

In [31]:
df2 = pd.read_csv('books.csv', on_bad_lines='skip')

# 5. Recommendation function using cosine similarity


In [32]:
def prepare_features(df2):
    # Create features from relevant columns
    features = pd.DataFrame()
    
    # Normalize numerical features
    df2.loc[ (df2['average_rating'] >= 0) & (df2['average_rating'] <= 1), 'rating_between'] = "between 0 and 1"
    df2.loc[ (df2['average_rating'] > 1) & (df2['average_rating'] <= 2), 'rating_between'] = "between 1 and 2"
    df2.loc[ (df2['average_rating'] > 2) & (df2['average_rating'] <= 3), 'rating_between'] = "between 2 and 3"
    df2.loc[ (df2['average_rating'] > 3) & (df2['average_rating'] <= 4), 'rating_between'] = "between 3 and 4"
    df2.loc[ (df2['average_rating'] > 4) & (df2['average_rating'] <= 5), 'rating_between'] = "between 4 and 5"
    
    # One-hot encode categorical features
    # Authors
    author_dummies = pd.get_dummies(df2['authors'], prefix='author')
    features = pd.concat([features, author_dummies], axis=1)
    
    # Publisher
    publisher_dummies = pd.get_dummies(df2['publisher'], prefix='publisher')
    features = pd.concat([features, publisher_dummies], axis=1)
    
    # Language
    language_dummies = pd.get_dummies(df2['language_code'], prefix='language')
    features = pd.concat([features, language_dummies], axis=1)
    
    return features.values

In [33]:
features = prepare_features(df2)

In [34]:
def get_recommendations(book_name, book_embeddings, df2, top_n=5):
    book_idx = df2[df2['title'] == book_name].index[0]
    book_embedding = book_embeddings[book_idx]
    
    # Calculate cosine similarity
    similarities = np.dot(book_embeddings, book_embedding) / (
        np.linalg.norm(book_embeddings, axis=1) * np.linalg.norm(book_embedding)
    )
    
    # Get top N similar books
    similar_indices = np.argsort(similarities)[::-1][1:top_n+1]
    
    return [(df2.iloc[idx]['title'], similarities[idx]) for idx in similar_indices]


In [35]:
# 6. Main execution
def setup_pytorch_recommender(features, df2):
    # Convert features to numpy array if it's not already
    if isinstance(features, pd.DataFrame):
        features = features.values
    
    # Create dataset and dataloader
    dataset = BookDataset(features)
    train_loader = DataLoader(dataset, batch_size=64, shuffle=True)
    
    # Initialize and train model
    input_size = features.shape[1]
    model = BookRecommenderNet(input_size)
    model = train_model(model, train_loader)
    
    # Get embeddings for all books
    book_embeddings = get_book_embeddings(model, features)
    
    return model, book_embeddings

In [37]:
# Setup the recommender
model, book_embeddings = setup_pytorch_recommender(features, df2)

Training started...


KeyboardInterrupt: 

In [22]:
def pytorch_book_recommender(book_name, book_embeddings, df2):
    recommendations = get_recommendations(book_name, book_embeddings, df2)
    return [title for title, _ in recommendations]

# Usage example:
if __name__ == "__main__":
    # Prepare features
    features = prepare_features(df2)
    
    print("Setting up PyTorch recommender system...")
    model, book_embeddings = setup_pytorch_recommender(features, df2)
    
    # Test the recommender
    test_book = 'Harry Potter and the Half-Blood Prince (Harry Potter  #6)'
    print(f"\nGetting recommendations for: {test_book}")
    recommendations = pytorch_book_recommender(test_book, book_embeddings, df2)
    
    print("\nRecommended Books:")
    for i, book in enumerate(recommendations, 1):
        print(f"{i}. {book}")

Setting up PyTorch recommender system...
Training started...
Epoch [10/100], Loss: 0.0002
Epoch [20/100], Loss: 0.0002
Epoch [30/100], Loss: 0.0002
Epoch [40/100], Loss: 0.0002
Epoch [50/100], Loss: 0.0002
Epoch [60/100], Loss: 0.0002
Epoch [70/100], Loss: 0.0002
Epoch [80/100], Loss: 0.0002
Epoch [90/100], Loss: 0.0002
Epoch [100/100], Loss: 0.0002
Training completed!

Getting recommendations for: Harry Potter and the Half-Blood Prince (Harry Potter  #6)

Recommended Books:
1. Harry Potter and the Order of the Phoenix (Harry Potter  #5)
2. Harry Potter and the Half-Blood Prince (Harry Potter  #6)
3. Harry Potter and the Chamber of Secrets (Harry Potter  #2)
4. The Journal of Scott Pendleton Collins: A World War 2 Soldier
5. Harry Potter and the Sorcerer's Stone (Harry Potter  #1)


In [23]:
# Get recommendations
test_book = 'Harry Potter and the Half-Blood Prince (Harry Potter  #6)'
recommendations = pytorch_book_recommender(test_book, book_embeddings, df2)